MA705 Final Project
LA Airbnb Data

In [4]:
# Load LA Airbnb Data
import pandas as pd
from pathlib import Path

data_path = Path("C:\\Users\\abbys\\OneDrive\\Desktop\\BENTLEY\\MA705\\~Final Project - Boston Airbnb\\Data\\LA_Airbnb_ALL.csv")

df = pd.read_csv(data_path)

#Inspect the dataset
print("Shape:", df.shape)
print("\nColumn names:\n", df.columns.tolist())
print("\nData types:\n", df.dtypes)

# Missing Values
missing = df.isna().sum().sort_values(ascending=False)
print("\nMissing values per column:\n", missing)

# Top 3 Missing Values
top3_missing = missing.head(3)
print("\nTop 3 columns with the most missingness:\n", top3_missing)

Shape: (45886, 79)

Column names:
 ['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name', 'description', 'neighborhood_overview', 'picture_url', 'host_id', 'host_url', 'host_name', 'host_since', 'host_location', 'host_about', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_listings_count', 'host_total_listings_count', 'host_verifications', 'host_has_profile_pic', 'host_identity_verified', 'neighbourhood', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude', 'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price', 'minimum_nights', 'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated', 'has_availability', 'availability_30', 'availability

In [14]:
# -----------------------------------------------------------
# ENRICHMENT 1: NEIGHBORHOOD-LEVEL AGGREGATES
# -----------------------------------------------------------

# Compute mean price, avg reviews per month, and listing count by neighborhood
neigh_stats = (
    df.groupby("neighbourhood")
      .agg(
            neighbourhood_avg_price=("price", "mean"),
            neighbourhood_avg_reviews=("reviews_per_month", "mean"),
            neighbourhood_listing_count=("id", "count")
      )
      .reset_index()
)

# Merge aggregated neighborhood stats back into the main dataset
df_enriched = df.merge(neigh_stats, on="neighbourhood", how="left")

print(df_enriched[[
    "id", "neighbourhood", "price",
    "neighbourhood_avg_price",
    "neighbourhood_listing_count"
]].head())


     id            neighbourhood  price  neighbourhood_avg_price  \
0  2708  Neighborhood highlights   81.0               414.677108   
1  2732  Neighborhood highlights  110.0               414.677108   
2  2864  Neighborhood highlights    NaN               414.677108   
3  6033                      NaN    NaN                      NaN   
4  6931  Neighborhood highlights   88.0               414.677108   

   neighbourhood_listing_count  
0                      19937.0  
1                      19937.0  
2                      19937.0  
3                          NaN  
4                      19937.0  


In [15]:
# -----------------------------------------------------------
# ENRICHMENT 2: HOST-LEVEL AGGREGATES
# -----------------------------------------------------------

host_stats = (
    df.groupby("host_id")
      .agg(
            host_total_listings=("id", "count"),
            host_avg_price=("price", "mean"),
            host_avg_reviews=("reviews_per_month", "mean")
      )
      .reset_index()
)

# Merge host statistics into enriched dataframe
df_enriched = df_enriched.merge(host_stats, on="host_id", how="left")

df_enriched[[
    "id", "host_id", "host_total_listings", "host_avg_price"
]].head()


id  host_id  host_total_listings  host_avg_price
0  2708     3008                    2            84.5
1  2732     3041                    2           160.0
2  2864     3207                    1             NaN
3  6033    11619                    6           216.0
4  6931     3008                    2            84.5

In [16]:
# -----------------------------------------------------------
# ENRICHMENT 3: EXTERNAL DATA MERGE EXAMPLE
# -----------------------------------------------------------

# Example external dataset (normally this would be read from CSV/API)
zip_income = pd.DataFrame({
    "zipcode": [90001, 90002, 90003, 90004, 90005],
    "median_income": [40120, 38500, 39200, 61200, 50800]
})

# If zipcode exists in your Airbnb dataset (check first)
if "zipcode" in df_enriched.columns:
    df_enriched = df_enriched.merge(zip_income, on="zipcode", how="left")


In [17]:
print("Final enriched dataframe shape:", df_enriched.shape)
df_enriched.head()


Final enriched dataframe shape: (45886, 85)


id                        listing_url       scrape_id last_scraped  \
0  2708  https://www.airbnb.com/rooms/2708  20250901181307   2025-09-03   
1  2732  https://www.airbnb.com/rooms/2732  20250901181307   2025-09-03   
2  2864  https://www.airbnb.com/rooms/2864  20250901181307   2025-09-03   
3  6033  https://www.airbnb.com/rooms/6033  20250901181307   2025-09-03   
4  6931  https://www.airbnb.com/rooms/6931  20250901181307   2025-09-03   

            source                                               name  \
0      city scrape  Run Runyon, Beaut Furn Mirror Mini-Suit w/ Fir...   
1      city scrape                              Zen Life at the Beach   
2  previous scrape             * Beautiful Master Suite/Jacuzzi Tub/*   
3  previous scrape                           Poolside Serenity Studio   
4      city scrape  RUN Runyon, Beau Furn Rms Terrace Hollyw Hill ...   

                                         description  \
0  Run Runyon Canyon<br /><br />Gym & Sauna <br /...   
1                An oasis of tranquility awaits you.   
2  Centrally located.... Furnished with King Size...   
3  Our distinctive bachelor's studio for 1-3 gues...   
4  Run Runyon Canyon & Views<br /><br />Gym & Sau...   

                               neighborhood_overview  \
0  Walk and run to Runyon Canyon, it is open!<br ...   
1  This is the best part of Santa Monica. Quiet, ...   
2  What makes the neighborhood unique is that the...   
3                                                NaN   
4  We are in the middle of one of the great citie...   

                                         picture_url  host_id  ...  \
0  https://a0.muscache.com/pictures/hosting/Hosti...     3008  ...   
1  https://a0.muscache.com/pictures/1082993/c5a99...     3041  ...   
2  https://a0.muscache.com/pictures/23817858/de20...     3207  ...   
3  https://a0.muscache.com/pictures/458111/986c76...    11619  ...   
4  https://a0.muscache.com/pictures/miso/Hosting-...     3008  ...   

  calculated_host_listings_count_entire_homes  \
0                                           0   
1                                           1   
2                                           1   
3                                           3   
4                                           0   

  calculated_host_listings_count_private_rooms  \
0                                            2   
1                                            1   
2                                            0   
3                                            3   
4                                            2   

  calculated_host_listings_count_shared_rooms reviews_per_month  \
0                                           0              0.33   
1                                           0              0.14   
2                                           0               NaN   
3                                           0              0.10   
4                                           0              0.19   

  neighbourhood_avg_price neighbourhood_avg_reviews  \
0              414.677108                  1.371826   
1              414.677108                  1.371826   
2              414.677108                  1.371826   
3                     NaN                       NaN   
4              414.677108                  1.371826   

  neighbourhood_listing_count host_total_listings host_avg_price  \
0                     19937.0                   2           84.5   
1                     19937.0                   2          160.0   
2                     19937.0                   1            NaN   
3                         NaN                   6          216.0   
4                     19937.0                   2           84.5   

  host_avg_reviews  
0            0.260  
1            0.205  
2              NaN  
3            0.264  
4            0.260  

[5 rows x 85 columns]